In [3]:
import os
from dotenv import load_dotenv
from agno.agent import Agent, RunResponse
from agno.models.openai.like import OpenAILike

load_dotenv()
api_key=os.getenv("QWEN_API_KEY")
base_url=os.getenv("QWEN_API_BASE_URL")
local_base_url = 'http://192.168.0.166:8000/v1'
local_model_name = 'Qwen3-235B'
model_name = 'qwen-plus-latest'

local_settings = {
  'api_key' : '123',
  'base_url' : local_base_url,
  'id' : local_model_name
}

qwen_settings = {
  'api_key' : api_key,
  'base_url' : base_url,
  'id' : model_name
}

settings = qwen_settings

In [4]:
from pydantic import BaseModel,Field

class Classification(BaseModel):
    sentiment:str = Field(description="The sentiment of the text.")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")
    

In [16]:
# Agent that uses JSON mode
json_mode_agent = Agent(
    model=OpenAILike(**settings),
    # description="You write movie scripts.",
    response_model=Classification,
    use_json_mode=True,
)

In [17]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
response = json_mode_agent.run(inp)

In [18]:
response.content

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

In [19]:
response.messages

[Message(role='system', content='Provide your output as a JSON containing the following fields:\n<json_fields>\n["sentiment", "aggressiveness", "language"]\n</json_fields>\n\nHere are the properties for each field:\n<json_field_properties>\n{\n  "sentiment": {\n    "description": "The sentiment of the text.",\n    "type": "string"\n  },\n  "aggressiveness": {\n    "description": "How aggressive the text is on a scale from 1 to 10",\n    "type": "integer"\n  },\n  "language": {\n    "description": "The language the text is written in",\n    "type": "string"\n  }\n}\n</json_field_properties>\nStart your response with `{` and end it with `}`.\nYour output will be passed to json.loads() to convert it to a Python object.\nMake sure it only contains valid JSON.', name=None, tool_call_id=None, tool_calls=None, audio=None, images=None, videos=None, files=None, audio_output=None, image_output=None, thinking=None, redacted_thinking=None, provider_data=None, citations=None, reasoning_content=None

In [ ]:
# Agent that uses structured outputs
structured_output_agent = Agent(
    model=OpenAILike(**settings),
    # description="Agent that uses structured outputs with json",
    response_model=Classification,
)

In [20]:
inp = """Extract the desired information from the following passage.

Only extract the properties mentioned in json.

Passage: Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"""
response = structured_output_agent.run(inp)

WARNING  Failed to parse cleaned JSON: 2 validation errors for Classification                                      
         aggressiveness                                                                                            
           Field required [type=missing, input_value={'sentiment': 'positivo',...y esperanza de amistad'},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.11/v/missing                              
         language                                                                                                  
           Field required [type=missing, input_value={'sentiment': 'positivo',...y esperanza de amistad'},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.11/v/missing

WARNING  Failed to parse as Python dict: 2 validation errors for Classification                                    
         aggressiveness                                                                                            
           Field required [type=missing, input_value={'sentiment': 'positivo',...y esperanza de amistad'},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.11/v/missing                              
         language                                                                                                  
           Field required [type=missing, input_value={'sentiment': 'positivo',...y esperanza de amistad'},         
         input_type=dict]                                                                                          
             For further information visit https://errors.pydantic.dev/2.11/v/missing

WARNING  Failed to convert response to response_model

In [15]:
response.messages

[Message(role='system', content='Agent that uses structured outputs with json', name=None, tool_call_id=None, tool_calls=None, audio=None, images=None, videos=None, files=None, audio_output=None, image_output=None, thinking=None, redacted_thinking=None, provider_data=None, citations=None, reasoning_content=None, tool_name=None, tool_args=None, tool_call_error=None, stop_after_tool_call=False, add_to_agent_memory=True, from_history=False, metrics=MessageMetrics(input_tokens=0, output_tokens=0, total_tokens=0, audio_tokens=0, input_audio_tokens=0, output_audio_tokens=0, cached_tokens=0, cache_write_tokens=0, reasoning_tokens=0, prompt_tokens=0, completion_tokens=0, prompt_tokens_details=None, completion_tokens_details=None, additional_metrics=None, time=None, time_to_first_token=None, timer=None), references=None, created_at=1749980117),
 Message(role='user', content='Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!', name=None, tool_call_id=None, to